# 统计方法与机器学习-实验报告4

温兆和 10205501432

## 背景描述

汽车发动机在测功机上产生的制动马力被认为是发动机转速(每分钟转数，$rpm$)、燃料的道路辛烷值和发动机压缩值的函数，我们在实验室里进行实验，研究它们的函数关系。

## 数据描述

|变量名|变量含义|变量类型|变量取值范围|
|------|-------|-------|-----------|
|rpm|发动机转速|连续变量|$R^{+}$|
|Road_Octane_Number|道路辛烷值|连续变量|$R^{+}$|
|Compression|压缩值|连续变量|$R^{+}$|
|Brake_Horsepower|制动马力|连续变量|$R^{+}$|

显著性水平$α$取$0.05$。

## 实验过程
- 请用多元线性回归模型，描述制动马力和发动机转速、道路辛烷值以及压缩值之间的函数关系。

首先，我们引入本次实验需要用到的Python库，并打开、处理数据集：

In [1]:
import os # 修改工作目录

import numpy as np
import pandas as pd
import scipy.stats as stats
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from jupyterquiz import display_quiz

import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm
from scipy.stats import f
from scipy.stats import t

from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score
from sklearn import preprocessing
from statsmodels.stats.stattools import durbin_watson

Data = pd.read_csv("./Project_3.csv")
print(Data.head())

    rpm  Road_Octane_Number  Compression  Brake_Horsepower
0  2000                  90          100               225
1  1800                  94           95               212
2  2400                  88          110               229
3  1900                  91           96               222
4  1600                  86          100               219


计算出本次实验的样本量、自变量数，并设置好显著性水平等参数：

In [2]:
alpha = 0.05
n = Data.shape[0]
p = Data.shape[1] - 1
print("The number of instances is ", n)
print("The number of features is ", p)

The number of instances is  12
The number of features is  3


在多元线性回归模型中$$y=\beta_{0}+\beta_{1}x_{1}+...+\beta_{p}x_{p}+\epsilon, 本实验中p=3$$
假设$$E(\epsilon)=0,Var(\epsilon)=\sigma^2<\infty$$
其中$y$为因变量，对应本实验中的制动马力；诸$x_{i}$为自变量，分别对应本实验中的发动机转速、燃料的道路辛烷值和发动机压缩值。当我们收集到$n$组样本时，第$i$组数据的数值模型可写为$$y_{i}=\beta_{0}+\beta_{1}x_{i1}+...+\beta_{p}x_{ip}+\epsilon_{i}, 本实验中p=3$$
把这个模型写成矩阵的形式，得
$$\mathbf{y} = \mathbf{X}\mathbf{\beta} + \mathbf{\epsilon}$$其中，因变量
$$\mathbf{y} = \begin{pmatrix}y_1\\y_2\\\vdots\\ y_n\end{pmatrix},本实验中n=12$$
自变量$$\mathbf{X} = \begin{pmatrix}
1 & x_{11} & x_{12} & x_{13}\\
1 & x_{21} & x_{22} & x_{23}\\
\vdots & \vdots & \vdots & \vdots\\
1 & x_{n1} & x_{n2} & x_{n3}\\
\end{pmatrix},本实验中n=12$$
待估参数$$\mathbf{\beta} = \begin{pmatrix}
\beta_0 \\ \beta_1 \\ \beta_2 \\ \beta_3
\end{pmatrix}$$
误差$$\mathbf{\epsilon} = \begin{pmatrix}\epsilon_1\\\epsilon_2\\\vdots\\ \epsilon_n\end{pmatrix},本实验中n=12$$
而待估参数$\mathbf{\beta}$的估计值与数据之间的关系为$$\hat{\mathbf{\beta}} = (\mathbf{X}'\mathbf{X})^{-1} \mathbf{X}'\mathbf{y}$$
由此，我们就可以得到制动马力和发动机转速、道路辛烷值以及压缩值之间的函数关系。我们只用Matrix Calculus方法。

In [4]:
Data1 = sm.add_constant(Data)
Data1_value = Data1.values
X = Data1_value[:,0:(p+1)]
y = Data1_value[:,-1]
beta_hat_1 = np.linalg.inv(X.T @ X) @ (X.T @ y)
# A @ B <=> np.dot(A,B) matrix multiply

print("The estimates of the parameters are \n", 
      np.around(beta_hat_1,4))

[[1.00e+00 2.00e+03 9.00e+01 1.00e+02 2.25e+02]
 [1.00e+00 1.80e+03 9.40e+01 9.50e+01 2.12e+02]
 [1.00e+00 2.40e+03 8.80e+01 1.10e+02 2.29e+02]
 [1.00e+00 1.90e+03 9.10e+01 9.60e+01 2.22e+02]
 [1.00e+00 1.60e+03 8.60e+01 1.00e+02 2.19e+02]
 [1.00e+00 2.50e+03 9.60e+01 1.10e+02 2.78e+02]
 [1.00e+00 3.00e+03 9.40e+01 9.80e+01 2.46e+02]
 [1.00e+00 3.20e+03 9.00e+01 1.00e+02 2.37e+02]
 [1.00e+00 2.80e+03 8.80e+01 1.05e+02 2.33e+02]
 [1.00e+00 3.40e+03 8.60e+01 9.70e+01 2.24e+02]
 [1.00e+00 1.80e+03 9.00e+01 1.00e+02 2.23e+02]
 [1.00e+00 2.50e+03 8.90e+01 1.04e+02 2.30e+02]]
The estimates of the parameters are 
 [-2.660312e+02  1.070000e-02  3.134800e+00  1.867400e+00]


于是，我们就可以通过数据集中的数据估计出制动马力和发动机转速、道路辛烷值以及压缩值之间的函数关系大致为$$y=-266.0312+0.0107x_{1}+3.1348x_{2}+1.8674x_{3}$$
我们再用scikit-learn方法来验算一下：

In [28]:
model2 = linear_model.LinearRegression()
X_without_intercept = X[:,1:4]
model2.fit(X_without_intercept, y)
beta_hat_3 = np.append(np.array(model2.intercept_),model2.coef_)
print("The estimates of the parameters are \n", 
      np.around(beta_hat_3,4))

The estimates of the parameters are 
 [-2.660312e+02  1.070000e-02  3.134800e+00  1.867400e+00]


和前一种方法得到的结果是一样的。
- 分别将数据中心化、标准化之后，比较参数估计的异同，并进行评述（提示：可以结合理论课的课件）。

数据的中心化是指将矩阵$\mathbf{X}$中的每一个$x_{ij}$和向量$\mathbf{y}$中的每一个$y_{i}$都对该列的平均值作差，即
$$x_{ij}^*=x_{ij}-\frac{1}{n}\sum_{i=1}^{n} x_{ij}$$
$$y_{i}^*=y_{i}-\frac{1}{n}\sum_{i=1}^{n} y_{i}$$
我们先对这个数据集中的数据进行中心化：

In [29]:
## 中心化
X_center = preprocessing.scale(X_without_intercept, with_mean = True, with_std=False)
y_center = preprocessing.scale(y, with_mean = True, with_std=False)
# with_mean = True (default), with_std = True (default)

# print(X_center) 

print("The sample means of centered features are ", np.around(np.mean(X_center,axis=0),4))
print("The sample mean of centered response is ", np.around(np.mean(y_center,axis=0),4))

The sample means of centered features are  [-0. -0.  0.]
The sample mean of centered response is  0.0


可见，中心化后$\mathbf{X}$、$\mathbf{y}$各列均值$\frac{1}{n}\sum_{i=1}^n x_{ij}^*$、$\frac{1}{n}\sum_{i=1}^n y_{i}^*$均为$0$，因为$$\sum_{i=1}^n x_{ij}^*=0$$
$$\sum_{i=1}^n y_{i}^*=0$$
这从$x_{ij}^*$、$y_{i}^*$的定义很容易看出来。
下面，我们再来计算出中心化后$\mathbf{\beta}$的估计值：

In [30]:
model3 = linear_model.LinearRegression()
model3.fit(X_center, y_center)
beta_hat_4 = np.append(np.array(model3.intercept_),model3.coef_)
print("The estimates of the parameters are \n", 
      np.around(beta_hat_4,4))

The estimates of the parameters are 
 [0.     0.0107 3.1348 1.8674]


可见，当数据被中心化之后，截距项$\beta_{0}$的估计值变为$0$，而斜率项$(\beta_{1},\beta_{2},\beta_{3})^T$的估计值和中心化之前相比没有变化。也就是说，中心化单纯只是消除了截距对响应变量带来的影响。

下面我们再来计算标准化数据集后$\mathbf{\beta}$的估计值。标准化是指，对每一个$x_{ij}$都作这样的处理：
$$x_{ij}^{**}=\frac{x_{ij}^*}{\sqrt{\sum_{i=1}^n (x_{ij}-\bar{x_{j}})^2}}$$
对每一个$y_{i}$都有
$$y_{i}^{**}=\frac{y_{i}^*}{\sqrt{\sum_{i=1}^n (y_{i}-\bar{y})^2}}$$

In [31]:
X_center_scaled = preprocessing.scale(X_without_intercept, with_mean = True, with_std=True)
y_center_scaled = preprocessing.scale(y, with_mean = True, with_std=True)
model1 = linear_model.LinearRegression()
model1.fit(X_center_scaled, y_center_scaled)
beta_hat_2 = np.append(np.array(model1.intercept_),model1.coef_)
print("The estimates of the parameters are \n", 
      np.around(beta_hat_2,4))

The estimates of the parameters are 
 [0.     0.3757 0.5793 0.5477]


我们可以发现，截距项$\beta_{0}$的估计值仍然是$0$，而在三个斜率项中，$\beta_{1}$的估计值相比原来变小了，而$\beta_{2}、\beta_{3}$的估计值相比原来变大了。但是在标准化之前，三个斜率项中$\beta_{2}$的估计值最大，$\beta_{3}$次之，$\beta_{1}$的估计值最小。在标准化之后三个斜率项之间的相对大小关系亦是如此。可见，标准化只是消除了变量的量纲对响应变量产生的影响，并不会影响到自变量本身对响应变量影响程度的大小。

- 从模型显著性、参数显著性以及残差分析三个角度，分析多元线性回归模型是否合理。

我们先来检验一下模型显著性和参数显著性。

模型的显著性是指，在$\beta_{0}、\beta_{1}、\beta_{2}、\beta_{3}$中是否存在至少一个不为零的数。如果它们都为零，则说明通过发动机转速、燃料的道路辛烷值和发动机压缩值来刻画、预测制动马力是不合适的。检验模型的显著性通常使用$F$检验。我们构造检验统计量
$$F_{0}=\frac{SS_R/p}{SS_E/(n-p-1)}$$
其中
$$SS_R=\sum_{i=1}^n (\hat{y_{i}}-\bar{y})^2$$
$$SS_E=\sum_{i=1}^n (y_{i}-\hat{y_{i}})^2$$
来检验
$$H_0: \beta_{1}=\beta_{2}=\beta_{3}=0 v.s. H_1: \exists i\in\{1,2,3\} s.t. \beta_{i}\neq0$$
由于当$H_0$成立时
$$F_0\sim F(p,n-p-1)$$
故在显著性水平$\alpha$下，拒绝域为
$$W=\{F_0>F_{1-\alpha}(p,n-p-1)\}$$
$p$值为
$$p=P(F\geqslant F_0)$$
但模型显著性检验只是检验了三个自变量中至少有一个对响应变量有影响。如果要找出三个自变量中对响应变量没有影响的并剔除之，我们还需要对参数显著性进行检验。检验参数显著性一般用$t$检验法来检验
$$H_{0j}: \beta_j=0 v.s. H_{1j}:\beta_j\neq 0, j\in\{1,2,3\} $$
如果原假设成立，我们就认为自变量$x_j$对响应变量没有影响；如果备择假设成立，我们就认为自变量$x_j$对响应变量有影响。
我们构造检验统计量
$$t_j=\frac{\hat{\beta_j}\sqrt{n-p-1}}{\sqrt{c_{jj}SS_E}}$$
其中$c_{jj}$表示矩阵$(\mathbf{X}^T\mathbf{X})^{-1}$中第$(j+1)$行，第$(j+1)$列$(j\in \{0,1,2,3\})$的元素。
由于
$$t_j\sim t(n-p-1)$$
所以显著性水平$\alpha$下拒绝域为
$$W=\{|t_j|\geqslant t_{1-\alpha/2}(n-p-1)\}$$
当$t_j$落入拒绝域，我们拒绝原假设，认为$\beta_j\neq 0$。
下面，我们就来计算上述检验中各检验统计量的值。

In [41]:
model4 = ols("Brake_Horsepower~rpm + Road_Octane_Number + Compression",Data).fit()
model4.summary()

/opt/anaconda3/envs/jh-ecnu/lib/python3.9/site-packages/scipy/stats/stats.py:1541: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=12
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:       Brake_Horsepower   R-squared:                       0.807
Model:                            OLS   Adj. R-squared:                  0.734
Method:                 Least Squares   F-statistic:                     11.12
Date:                Thu, 29 Sep 2022   Prob (F-statistic):            0.00317
Time:                        21:28:25   Log-Likelihood:                -40.708
No. Observations:                  12   AIC:                             89.42
Df Residuals:                       8   BIC:                             91.36
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
======================================================================================
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
Intercept           -266.0312     92.674     -2.871      0.021    -479.737     -52.325
rpm                    0.0107      0.004      2.390      0.044       0.000       0.021
Road_Octane_Number     3.1348      0.844      3.712      0.006       1.188       5.082
Compression            1.8674      0.535      3.494      0.008       0.635       3.100
==============================================================================
Omnibus:                        0.392   Durbin-Watson:                   1.043
Prob(Omnibus):                  0.822   Jarque-Bera (JB):                0.230
Skew:                          -0.282   Prob(JB):                        0.891
Kurtosis:                       2.625   Cond. No.                     9.03e+04
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 9.03e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

可见，$F$检验的$p$值为$0.00317$，小于给定的显著性水平$\alpha=0.05$，所以我们拒绝原假设，认为这个线性回归模型是有意义的。而发动机转速、燃料的道路辛烷值和发动机压缩对应的$t$检验的$p$值分别为$0.044、0.006、0.008$,均小于该显著性水平。所以我们拒绝原假设，认为这三个自变量对响应变量（制动马力）的影响都是显著的。

最后，我们再来验证这个模型的正态假设是否成立，即
$$\epsilon_i\sim N(0,\sigma^2),i\in [1,12]\bigcap Z$$
且诸$\epsilon_i$相互独立。我们还是使用第一周实验中使用的方法来分别检验残差的独立性、方差齐性和正态性。具体的原理不再赘述，我们只分析结果、得出结论。

我们先计算出各组样本的残差：

In [35]:
Data_values = Data.values
data_res = Data_values.astype(float)
Data["residuals"]=-Data["Brake_Horsepower"]+beta_hat_1[0]+beta_hat_1[1]*Data["rpm"]+beta_hat_1[2]*Data["Road_Octane_Number"]+beta_hat_1[3]*Data["Compression"]
print("The residuals are as follows: \n", Data["residuals"])

The residuals are as follows: 
 0     -0.731289
1     13.328247
2     11.958476
3     -3.137442
4    -11.555798
5    -10.891754
6     -2.213675
7      0.124560
8      2.906712
9     -2.874252
10    -0.873931
11     3.960146
Name: residuals, dtype: float64


接着，我们用Durbin-Watson方法检验误差的独立性。

In [36]:
DW = durbin_watson(Data["residuals"])
print("DW statistic is", round(DW,4))

DW statistic is 1.0431


$DW$值为$1.0431$，无法确定残差之间是否独立。

接着，我们用Shapiro-Wilk方法检验误差的正态性。

In [39]:
SW_stat,SW_pVal = stats.shapiro(Data["residuals"])
print("Shapiro-Wilk test statistic is", round(SW_stat,4))
print("The p value is", round(SW_pVal,4))

Shapiro-Wilk test statistic is 0.9291
The p value is 0.3706


由于$p$值大于显著性水平$\alpha=0.05$，我们接受原假设，认为残差是服从正态分布的。由于只有一组残差，我们不再验证方差齐性。只要这一组残差服从正态分布，它们就一定方差相等。

所以，我们无法认为这个线性回归模型的残差符合正态性假设。

综上所述，我们从模型显著性、参数显著性两个角度分析了这个多元线性回归模型是合理的，但无法确认残差是否符合独立正态假设。

- 若取发动机转速为$3000r/min$，道路辛烷值为$90$，发动机压缩值为$100$时，分别给出制动马力值的置信区间和预测区间。

在多元线性回归模型中，响应变量的点预测值就等于它在该点的估计值
$$\hat{y_0}=\mathbf{x_0^T}(\mathbf{X^TX})^{-1}\mathbf{X^Ty}$$
预测区间为
$$
\hat{y}_0 \pm t_{1-\alpha/2}(n-p-1) \hat{\sigma} \sqrt{1+\mathbf{x}_0' (\mathbf{X}'\mathbf{X})^{-1} \mathbf{x}_0}
$$
其中$$\hat{\sigma}^2 = (n-p-1) \sum_{i=1}^n e_i^2$$
我们来根据这个公式计算制动马力的点预测和区间预测：

In [45]:
def prediction_interval(x0,X,y):
    # Add intercept to the new vector
    x0 = np.append(1,x0)
    # Parameter setting
    n = X.shape[0]
    p = X.shape[1]-1
    # Modelling
    beta_hat = np.linalg.inv(X.T @ X) @ (X.T @ y) # parameter estimation
    y_fitted = X @ beta_hat # fitted value
    e = y_fitted - y # residuals
    sigma2 = sum(e**2)/(n - p - 1) # estimate of sigma2
    # Interval Construction
    tVal = t.ppf(1-alpha/2, n- p - 1) # quantile
    delta = tVal*np.sqrt(sigma2)*np.sqrt(1 + x0.T @ np.linalg.inv(X.T @ X) @ x0)
    y0_fitted = x0 @ beta_hat
    output = [y0_fitted - delta, y0_fitted + delta] # prediction interval
    return output
print("制动马力的点预测为", np.around(model2.predict(np.array([[3000,90,100]])),4))
print("制动马力的区间预测为", np.around(prediction_interval(np.array([[3000,90,100]]),X,y),4))

制动马力的点预测为 [234.9819]
制动马力的区间预测为 [212.8622 257.1016]


也就是说，在这个自变量条件下制动马力最有可能接近$234.9819$，落入区间$[212.8622,257.1016]$的概率至少为$1-\alpha=0.95$。